In [64]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pickle
import datetime

In [2]:
def name_formatter(name):
    name = re.sub("_", " ", name).strip().title()
    return(name)

In [3]:
def ar_lookup(name):
    """
    Convert names from English to Arabic using European Media Monitors and 
     code modified from Phil Schrodt.
    """
    name = name_formatter(name)
    base_url = "http://emm.newsexplorer.eu/NewsExplorer/search/en/entities?query="
    url = base_url + name
    
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "lxml")
        name_url = soup.find("p", {"class" : "center_headline"}).find("a")['href']
    except Exception as e:
        print("Couldn't get page of results back: ", e)
        return []

    try:
        base = "http://emm.newsexplorer.eu/NewsExplorer/search/en/"
        name_url = base + name_url
        name_page = requests.get(name_url)
        soup = BeautifulSoup(name_page.content, "lxml")
        # check to make sure in list???? Take in alt names???/petrarch2/petrarch2/data/dictionaries
        names = soup.find("td", {"colspan" : "1"}).find_all("p")
        names = [i.text for i in names][1:]
        names_en = [i for i in names if re.search("\(.*?Eu|\(.*?en", i)]
        names_en = [re.sub("\s+?\(.+?\)", "", name) for name in names_en]
        print("Found match. Matched English name: ", names_en[0])
        names_ar = [i for i in names if re.search("\(.*?ar", i)]
        names_ar = [re.sub("\s+?\(.+?\)", "", name) for name in names_ar]
import pickle
im
        return names_ar
    except Exception:
        traceback.print_exc()
        return []

In [4]:
ar_lookup("ANGELA_MERKEL")

Found match. Matched English name:  Angela Merkel


['أنجيلا ميركل', 'أنجيلا ميركيل']

 ##Dictionary loading part

In [6]:
def clean_line(line):
    # Take out extra space, underscores, comments, etc.
    cleaned = re.sub("_* .+", "", line).strip()
    cleaned = re.sub("_$", "", cleaned, flags=re.MULTILINE)
    return cleaned

def ingest_dictionary(dict_path):
    """
    Read in the country (or other) actor dictionaries.
    """
    with open(dict_path) as f:
        country_file = f.read()
    split_file = country_file.split("\n")
    
    dict_dict = []
    key_name = ""
    alt_names = [] 
    roles = []

    for line in split_file:
        if not line:
            pass
        elif line[0] == "#":
            pass
        elif re.match("[A-Z]", line[0]):
            # handle the previous
            entry = {"actor_en" : key_name,
                    "alt_names_en" : alt_names,
                    "roles" : roles}
            dict_dict.append(entry)
            # zero everything out
            alt_names = []
            roles = []
            # make new key name
            key_name = clean_line(line)
            # check to see if the role is built in
            if bool(re.search("\[[A-Z]{3}\]", line)):
                roles = re.findall("\[(.+?)\]", line)
        elif line[0] == "+":
            cleaned = clean_line(line[1:])
            alt_names.append(cleaned)
        elif re.match("\s", line):
            roles.append(line.strip())
    return dict_dict 
dp = "./Phoenix.Countries.actors.txt"
dict_dict = ingest_dictionary(dp)

In [7]:
len(dict_dict)

18390

In [8]:
dict_dict[7777]

{'actor_en': 'AHMAD_CHALABI',
 'alt_names_en': [],
 'roles': ['[IRQELI 620101-030901]',
  '[IRQGOV 030901-030930]',
  '[IRQGOV 031101-040630]',
  '[IRQGOV 050601-060531]',
  '[IRQELI]']}

In [9]:
ar_lookup("AHMAD_CHALABI")

Found match. Matched English name:  Ahmad Chalabi


['أحمد الجلبي', 'أحمد شلبي', 'أحمد جلبي']

In [10]:
def eng_to_ar(dict_dict):
    """
    Update an English language dictionary entry with Arabic names
    """
    ar_names = ar_lookup(dict_dict["actor_en"])
    if not ar_names:
        print("No match found.")
        return dict_dict
    dict_dict['actor_ar'] = ar_names[0]
    if len(ar_names) > 1:
        dict_dict['alt_names_ar'] = ar_names[1:]
    return dict_dict

In [174]:
test=eng_to_ar(dict_dict[7777])

Found match. Matched English name:  Ahmad Chalabi


import pickle
with open('countrycode.pkl', 'wb') as output:
    pickle.dump(countrycode,output)

In [188]:
dic=pickle.load(open('countrycode.pkl', 'rb'))

In [189]:
countrycode=[key for key,value in dic.items()]

#Format stored in our db.
{ "_id" : ObjectId("5772026ca78a30ce5acbec43"), 
 "sentenceId" : "5771f20f0dfcd69f645102fa", 
 "word" : "President Obama", 
 "countryCode" : "USA", 
 "firstRoleCode" : "GOV", 
 "secondRoleCode" : "LEG", 
 "dateStart" : "Sun Jun 01 2008 00:00:00 GMT-0500 (CDT)", 
 "dateEnd" : "Wed Jun 01 2016 00:00:00 GMT-0500 (CDT)", 
 "confidenceFlag" : true, 
 "userId" : "577201cea78a30ce5acbec41", 
 "userName" : "guest", 
 "taggingTime" : ISODate("2016-06-28T04:51:56.092Z"),
 "__v" : 0 
 }

In [175]:
test

{'actor_ar': 'أحمد الجلبي',
 'actor_en': 'AHMAD_CHALABI',
 'alt_names_ar': ['أحمد شلبي', 'أحمد جلبي'],
 'alt_names_en': [],
 'roles': ['[IRQELI 620101-030901]',
  '[IRQGOV 030901-030930]',
  '[IRQGOV 031101-040630]',
  '[IRQGOV 050601-060531]',
  '[IRQELI]']}

In [ ]:
from pymongo import MongoClient
client=MongoClient()
client=MongoClient('mongodb://portland.cs.ou.edu:port/')
db=client['lexisnexis']
db.authenticate('username', 'password')
secondroles=db.secondroles
firstroles=db.agents
secondrolelist=[]
for item in secondroles.find():
    secondrolelist.append(item['id'])
firstrolelist=[]
for item in firstroles.find():
    firstrolelist.append(item['id'])

In [176]:
def transferStringToDateTime(inputtime):
    if(str(inputtime)!=''):
        dd=datetime.datetime.strptime(inputtime,'%y%m%d').date()
            #this is for 62 return 1962 not 2062:
        if  dd.year>2017:
            new=dd.replace(year=dd.year-100)
            return new
        else:
            return dd  
    else:
        return ''

In [177]:
transferStringToDateTime("620101")

datetime.date(1962, 1, 1)

In [178]:
#if it is first role it will return 1, if it is in secondRole list it will reture 2, and it it does not exist in both it return 3
def checkBelongToWhichRole(code):
    if code in firstrolelist:
        return 1;
    elif code in secondrolelist:
        return 2;
    else:
        return 3;

In [179]:
def checkIfCountrycodeExist(code):
    if code in countrycode:
        return 1
    else:
        return 0

In [180]:
#extract the roles and parse the string of each role to get the role format we like for our fajita website
def extract_actor_roles(dict):
    roles=[]
    word=dict['actor_ar']
    for item in dict['roles']:
        temp=[]
        country=item[1:4]
        temp.append(country)
        temp.append(checkIfCountrycodeExist(country))
        role=item[4:7]
        temp.append(role)
        temp.append(checkBelongToWhichRole(role))    
        temp.append(transferStringToDateTime(item[8:14]))
        temp.append(transferStringToDateTime(item[15:21]))
        temp.append(word)
        roles.append(temp)
    return roles

In [181]:
midresult=extract_actor_roles(test)
#from here each element in the list will build a dictionary entry in the mongo db

In [182]:
midresult

[['IRQ',
  1,
  'ELI',
  2,
  datetime.date(1962, 1, 1),
  datetime.date(2003, 9, 1),
  'أحمد الجلبي'],
 ['IRQ',
  1,
  'GOV',
  1,
  datetime.date(2003, 9, 1),
  datetime.date(2003, 9, 30),
  'أحمد الجلبي'],
 ['IRQ',
  1,
  'GOV',
  1,
  datetime.date(2003, 11, 1),
  datetime.date(2004, 6, 30),
  'أحمد الجلبي'],
 ['IRQ',
  1,
  'GOV',
  1,
  datetime.date(2005, 6, 1),
  datetime.date(2006, 5, 31),
  'أحمد الجلبي'],
 ['IRQ', 1, 'ELI', 2, '', '', 'أحمد الجلبي']]

In [183]:
#generate a list of json object for each element in the list
def generateFajitaObjectsList(midresult):
    finalResult=[]
    for item in midresult:
        tempjson={}
        #insert in all the deault stuff first
        tempjson["sentenceId"]="10000"
        tempjson["userName"]="robot"
        tempjson["userId"]="robot"
        tempjson["countryCode"]=item[0]
        tempjson["firstRoleCode"]=""
        tempjson["secondRoleCode"]=""
        tempjson["taggingTime"]= datetime.datetime.now().strftime('%Y-%m-%d')
        tempjson["dateStart"]=str(item[4])
        tempjson["dateEnd"]=str(item[5])
        tempjson["word"]=item[6]
        if(item[3]==1):
            tempjson["firstRoleCode"]=item[2]
        elif(item[3]==2):
            tempjson["secondRoleCode"]=item[2]
        else:
            pass
        finalResult.append(tempjson)
    return finalResult

In [184]:
generateFajitaObjectsList(midresult)

[{'countryCode': 'IRQ',
  'dateEnd': '2003-09-01',
  'dateStart': '1962-01-01',
  'firstRoleCode': '',
  'secondRoleCode': 'ELI',
  'sentenceId': '10000',
  'taggingTime': '2017-06-23',
  'userId': 'robot',
  'userName': 'robot',
  'word': 'أحمد الجلبي'},
 {'countryCode': 'IRQ',
  'dateEnd': '2003-09-30',
  'dateStart': '2003-09-01',
  'firstRoleCode': 'GOV',
  'secondRoleCode': '',
  'sentenceId': '10000',
  'taggingTime': '2017-06-23',
  'userId': 'robot',
  'userName': 'robot',
  'word': 'أحمد الجلبي'},
 {'countryCode': 'IRQ',
  'dateEnd': '2004-06-30',
  'dateStart': '2003-11-01',
  'firstRoleCode': 'GOV',
  'secondRoleCode': '',
  'sentenceId': '10000',
  'taggingTime': '2017-06-23',
  'userId': 'robot',
  'userName': 'robot',
  'word': 'أحمد الجلبي'},
 {'countryCode': 'IRQ',
  'dateEnd': '2006-05-31',
  'dateStart': '2005-06-01',
  'firstRoleCode': 'GOV',
  'secondRoleCode': '',
  'sentenceId': '10000',
  'taggingTime': '2017-06-23',
  'userId': 'robot',
  'userName': 'robot',
  